In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


### My plan

- Read dataset. Convert y into 0,1
- Create vocabulory of dataset
- Make each X value into a one hot of vocab vector. Maybe, add counts to it too
- Train naive bayes on it
- Make it better

In [2]:
df = pd.read_csv('/home/arjun/Desktop/Datasets/spam.csv', encoding='ISO-8859-1')

In [3]:
df = df[['v1','v2']]

In [4]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df['v1'] = df['v1'].map({'ham':0., 'spam':1.})
len(df)

5572

# Create vocab

In [6]:
vocab = {}
delimiters = re.compile(r'[ ,."\'?!:()]')
porter = PorterStemmer()
ct = 0
for i in range(len(df)):
    # print(df.iloc[i,1])
    words = delimiters.split(df.iloc[i,1].lower())
    # print(words)
    for word in words:
        if word.isalpha() and porter.stem(word) not in vocab:
            vocab[porter.stem(word)] = ct
            # print(porter.stem(word), ct)
            ct += 1
vocab_len = len(vocab)
vocab_len, ct


(6059, 6059)

In [7]:
df.head()

,v1,v2
0,0.0,"Go until jurong point, crazy.. Available only ..."
1,0.0,Ok lar... Joking wif u oni...
2,1.0,Free entry in 2 a wkly comp to win FA Cup fina...
3,0.0,U dun say so early hor... U c already then say...
4,0.0,"Nah I don't think he goes to usf, he lives aro..."


# One hot encoding

In [8]:
porter = PorterStemmer()
delimiters = re.compile(r'[ ,."\'?!:()]')
l = []
ct = 0
for r,c in df.iterrows():
    # print(r)
    one_hot = np.zeros(vocab_len)
    words = delimiters.split(c['v2'].lower())
    for word in words:
        # print(word)
        # print(vocab[word])
        if word.isalpha() and word in vocab:
            one_hot[vocab[word]] += 1
    l.append(one_hot)
    ct += 1
    
    # break
df['text'] = l
df = df.drop('v2', axis=1)
df = df.rename(columns={'v1':'label'})
len(l), ct

(5572, 5572)

In [9]:
df.head(10)

,label,text
0,0.0,"[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
1,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Training

In [10]:
df = np.array(df)

In [11]:
df[:,1]

array([array([1., 1., 1., ..., 0., 0., 0.]),
       array([0., 0., 0., ..., 0., 0., 0.]),
       array([0., 0., 0., ..., 0., 0., 0.]), ...,
       array([0., 0., 0., ..., 0., 0., 0.]),
       array([0., 0., 0., ..., 0., 0., 0.]),
       array([0., 0., 0., ..., 0., 0., 0.])], dtype=object)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df[:,1], df[:,0], test_size=0.2)


In [14]:
# X_train = X_train.reshape(-1,1)
X_train.shape, y_train.shape

((4457,), (4457,))